In [ ]:
"""
This script is designed to prevent the usage of Activity-Browser.

Author: Ao Chen. Place: UZH Zentrum. 
"""

In [1]:
import os
import pandas as pd
from brightway2 import *
from datetime import date

In [6]:
def project_setup(Project_name, Dataset_path, Dataset_lcia_path, Database_name):
    projects.set_current(Project_name) 
    if len(Database("biosphere3")) == 0: # if no biosphere3 database, set it up.
        bw2setup()
    if len(Database(Database_name)) == 0:
        ei38cutoff = SingleOutputEcospold2Importer(
            Dataset_path,
            Database_name
        ) # import ecoinvent database
        ei38cutoff.apply_strategies()
        ei38cutoff.statistics()
        ei38cutoff.write_database()
    if len(methods) == 0:
        ei38cutofflcia = SingleOutputEcospold2Importer(
            Dataset_lcia_path,
            Database_name
        )
        ei38cutofflcia.apply_strategies()
        ei38cutofflcia.statistics()
        ei38cutofflcia.write_database()

In [7]:
Project_name = "Ecoinvent" # if not exist, automatically create one
Dataset_path = "/Users/ocean/Library/CloudStorage/OneDrive-forpersonal/RA-SustainableConstruction/bwLCA/ecoinvent 3.8_cutoff_ecoSpold02/datasets"
Dataset_lcia_path = "/Users/ocean/Library/CloudStorage/OneDrive-forpersonal/RA-SustainableConstruction/bwLCA/ecoinvent 3.8_cutoff_lcia_ecoSpold02/datasets"

# Dataset_path = "/Users/ocean/Library/CloudStorage/OneDrive-forpersonal/RA-SustainableConstruction/bwLCA/ecoinvent 3.8_cutoff_ecoSpold02/test_dataset"
# Dataset_lcia_path = "/Users/ocean/Library/CloudStorage/OneDrive-forpersonal/RA-SustainableConstruction/bwLCA/ecoinvent 3.8_cutoff_lcia_ecoSpold02/test_dataset"
# substitute this with your ecoinvent datasets path (absolute path is more reliable)
Database_name = "cutoff38"
project_setup(Project_name, Dataset_path, Dataset_lcia_path, Database_name)

Extracting XML data from 19565 datasets
{http://www.EcoInvent.org/EcoSpold02}impactIndicator
{http://www.EcoInvent.org/EcoSpold02}impactIndicator
{http://www.EcoInvent.org/EcoSpold02}impactIndicator
{http://www.EcoInvent.org/EcoSpold02}impactIndicator
{http://www.EcoInvent.org/EcoSpold02}impactIndicator
{http://www.EcoInvent.org/EcoSpold02}impactIndicator
{http://www.EcoInvent.org/EcoSpold02}impactIndicator{http://www.EcoInvent.org/EcoSpold02}impactIndicator

{http://www.EcoInvent.org/EcoSpold02}impactIndicator
{http://www.EcoInvent.org/EcoSpold02}impactIndicator
{http://www.EcoInvent.org/EcoSpold02}impactIndicator
{http://www.EcoInvent.org/EcoSpold02}impactIndicator
{http://www.EcoInvent.org/EcoSpold02}impactIndicator
{http://www.EcoInvent.org/EcoSpold02}impactIndicator
{http://www.EcoInvent.org/EcoSpold02}impactIndicator
{http://www.EcoInvent.org/EcoSpold02}impactIndicator


In [ ]:
def calculation(Database_name, Material_list_path, Impact_method):
    db = Database(Database_name)
    process_list = db.search("*", limit=10000000) # read all processes in the database
    material_df = pd.read_excel(Material_list_path, header=None)
    material_df[['Activity', 'Product', 'Cutoff']] = material_df[0].str.split('|', 2, expand=True)
    material_df['Activity'] = material_df['Activity'].str.strip()

    CC_method = [m for m in methods if 'IPCC 2013' in str(m) and 'climate change' in str(m) and 'GWP 100a' in str(m) and not 'no LT' in str(m)][0]
    results = {}

    if not os.path.exists('calc_results_GWP100a'):
        os.makedirs('calc_results_GWP100a')

    for i in range(len(material_df)):
        target_process_list = [p for p in process_list if material_df['Activity'][i] in str(p)] # search for processes with specific name
        for j in range(len(target_process_list)):
            functional_unit = {target_process_list[j]: 1}
            lca = LCA(functional_unit, CC_method)
            lca.lci()
            lca.lcia()
            results[dict(target_process_list[j])['location']] = lca.score
        results_Series = pd.Series(results, name='IPCC_2013_GWP100a[kg CO2-Eq]')
        results_Series.to_csv('calc_results_GWP100a/Summary_'+material_df['Activity'][i]+'.csv')

In [ ]:
db = Database(Database_name)
Process_list = db.search("*", limit=10000000) # read all processes in the database
# TODO: this process is time consuming, run once is enough!

In [ ]:
Material_list_path = 'List_of_Materials_ecoinvent.xlsx'
Impact_method = [m for m in methods if 'IPCC 2013' in str(m) and 'climate change' in str(m) and 'GWP 100a' in str(m) and not 'no LT' in str(m)][0]
Today = date.today().strftime("%b_%m_%Y")
Result_path = Today+'_calc_results_GWP100a'
lca_calculation(Process_list, Material_list_path, Impact_method, Result_path)
# TODO: change names above as you need